In [1]:
import pandas as pd
import numpy as np

In [83]:
df = pd.read_csv('train_spam.csv')
df_test = pd.read_csv('test_spam.csv')



In [84]:
df.shape

(16278, 2)

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16278 entries, 0 to 16277
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text_type  16278 non-null  object
 1   text       16278 non-null  object
dtypes: object(2)
memory usage: 254.5+ KB


In [86]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4070 entries, 0 to 4069
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    4070 non-null   object
dtypes: object(1)
memory usage: 31.9+ KB


In [87]:
df.head()

,text_type,text
0,ham,make sure alex knows his birthday is over in f...
1,ham,a resume for john lavorato thanks vince i will...
2,spam,plzz visit my website moviesgodml to get all m...
3,spam,urgent your mobile number has been awarded wit...
4,ham,overview of hr associates analyst project per ...


In [88]:
df['text_type'].value_counts()

ham     11469
spam     4809
Name: text_type, dtype: int64

In [89]:
df.groupby('text_type').describe()

text                                                            \
           count unique                                                top   
text_type                                                                    
ham        11469  11458  SPAM ALERT  🚔  User:     Username: @DillyBubbl...   
spam        4809   4809  plzz visit my website moviesgodml to get all m...   

                
          freq  
text_type       
ham          7  
spam         1

In [90]:
# Convert every row to lower register
df['text'] = df['text'].apply(lambda x: x.lower())
df_test['text'] = df_test['text'].apply(lambda x: x.lower())

In [91]:
df['text']

0        make sure alex knows his birthday is over in f...
1        a resume for john lavorato thanks vince i will...
2        plzz visit my website moviesgodml to get all m...
3        urgent your mobile number has been awarded wit...
4        overview of hr associates analyst project per ...
                               ...                        
16273    if you are interested in binary options tradin...
16274    dirty pictureblyk on aircel thanks you for bei...
16275    or you could do this g on mon 1635465 sep 1635...
16276    insta reels par 80 गंद bhara pada hai 👀 kuch b...
16277    alex s paper comments 1 in the sentence betwee...
Name: text, Length: 16278, dtype: object

### ПРЕДОБРАБОТКА ТЕКСТА СООБЩЕНИЙ

In [92]:
import re
from nltk.stem import WordNetLemmatizer
import sklearn
import codecs
import pymorphy2
import seaborn as sns
from nltk.stem.snowball import SnowballStemmer

In [93]:
df

,text_type,text
0,ham,make sure alex knows his birthday is over in f...
1,ham,a resume for john lavorato thanks vince i will...
2,spam,plzz visit my website moviesgodml to get all m...
3,spam,urgent your mobile number has been awarded wit...
4,ham,overview of hr associates analyst project per ...
...,...,...
16273,spam,if you are interested in binary options tradin...
16274,spam,dirty pictureblyk on aircel thanks you for bei...
16275,ham,or you could do this g on mon 1635465 sep 1635...
16276,ham,insta reels par 80 गंद bhara pada hai 👀 kuch b...


In [94]:
df['text_type'] = df['text_type'].replace({'ham':0,'spam':1})

In [95]:
from nltk.corpus import stopwords
stopWords = set(stopwords.words('russian'))

def df_preprocess(text):    
    reg = re.compile('[^a-zA-Z0-9 ]') 
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'cite', text)
    text = re.sub('@[^\s]+', 'user', text)
    text = reg.sub(' ', text)
    
    # Лемматизация
    # morph = pymorphy2.MorphAnalyzer()
    # text =[morph.parse(word)[0].normal_form for word in text.split()]

    # Стемминг
    # stemmer = SnowballStemmer("english")
    # text =[stemmer.stem(word) for word in text.split()]

    #Стемминг с удалением стопслов
    stemmer = SnowballStemmer("english")
    text = ' '.join([stemmer.stem(word) for word in text.split() if word not in stopWords])

    return text

In [96]:
#Проверка лемматизации на 1м примере
print(df['text'].iloc[0])
%time print(df_preprocess(df['text'].iloc[0]))

make sure alex knows his birthday is over in fifteen minutes as far as youre concerned
make sure alex know his birthday is over in fifteen minut as far as your concern
CPU times: total: 0 ns
Wall time: 1.28 ms


In [97]:
# предобработка текста и лемматизация датасета потребовала бы +- 48 минут

In [98]:
#Проверка стемминга на 1м примере
print(df['text'].iloc[0])
%time print(df_preprocess(df['text'].iloc[0]))

make sure alex knows his birthday is over in fifteen minutes as far as youre concerned
make sure alex know his birthday is over in fifteen minut as far as your concern
CPU times: total: 0 ns
Wall time: 1.13 ms


In [99]:
# предобработка текста и стемминг датасета потребовала бы меньше минуты

In [100]:
#Стемминг с удалением стоп-слов займет меньше минуты времени

In [101]:
df['text'] = df['text'].apply(df_preprocess)

In [102]:
df.head()

,text_type,text
0,0,make sure alex know his birthday is over in fi...
1,0,a resum for john lavorato thank vinc i will ge...
2,1,plzz visit my websit moviesgodml to get all mo...
3,1,urgent your mobil number has been award with a...
4,0,overview of hr associ analyst project per davi...


In [103]:
df_test['text'] = df_test['text'].apply(df_preprocess)

In [104]:
X_train = df['text']
X_test = df_test['text']
y_train = df['text_type']

### Использую векторизатор BOW с биграмами

In [134]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1, 2))

X_train_BOW_bi = vectorizer.fit_transform(X_train)
X_test_BOW_bi = vectorizer.transform(X_test)

print(X_train_BOW_bi.shape, X_test_BOW_bi.shape)
     

(16278, 334737) (4070, 334737)


### Использую векторизатор TF-IDF с биграмами

In [135]:
from sklearn.feature_extraction.text import  TfidfVectorizer
vectorizer_TFIDF = TfidfVectorizer(ngram_range=(1,2))

X_train_TFIDF_bi = vectorizer_TFIDF.fit_transform(X_train)
X_test_TFIDF_bi = vectorizer_TFIDF.transform(X_test)

print(X_train_TFIDF_bi.shape, X_test_TFIDF_bi.shape)

(16278, 334737) (4070, 334737)


### Сделаю отбор фичей для каждого типа векторизации

In [137]:
from sklearn.feature_selection import SelectFromModel

from sklearn.svm import LinearSVC

In [144]:
lsvc = LinearSVC(C = .5,max_iter=10000) # C = 0.5
selective_model = SelectFromModel(lsvc,  max_features  = None)

In [145]:
X_train_BOW_bi_select_features = selective_model.fit_transform(X_train_BOW_bi,y_train)
X_test_BOW_bi_select_features = selective_model.transform(X_test_BOW_bi)
print('New shapes: ', X_train_BOW_bi_select_features.shape, X_test_BOW_bi_select_features.shape)


New shapes:  (16278, 66155) (4070, 66155)


In [146]:
X_train_TF_IDF_bi_select_features = selective_model.fit_transform(X_train_TFIDF_bi,y_train)
X_test_TF_IDF_bi_select_features = selective_model.transform(X_test_TFIDF_bi)
print('New shapes: ', X_train_TF_IDF_bi_select_features.shape, X_test_TF_IDF_bi_select_features.shape)

New shapes:  (16278, 102112) (4070, 102112)


## Теперь перехожу к обучению моделей

In [147]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression

In [148]:
cv = cross_validate(LogisticRegression(max_iter=1000),X_train_BOW_bi_select_features,y_train,cv=5,scoring=['accuracy','f1','roc_auc'])

### Проверка работоспособности модели на базовой логистической регрессии

In [149]:
print(f'Accuracy: {np.mean(cv["test_accuracy"])}')
print(f'f1: {np.mean(cv["test_f1"])}')
print(f'roc_auc: {np.mean(cv["test_roc_auc"])}')

Accuracy: 0.9443421196647004
f1: 0.8999804864383044
roc_auc: 0.9785849515191487


In [150]:
LogisticRegression(max_iter=1000).get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 1000,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

### Даже "из коробки" логистическая регрессия показывает достаточно высокие результаты

#### Попробую обучить модель на данных, векторизованных с помощью TF-IDF

In [151]:
cv_tf_idf = cross_validate(LogisticRegression(max_iter=1000),X_train_TF_IDF_bi_select_features,y_train,cv=5,scoring=['accuracy','f1','roc_auc'])

### Проверка работоспособности модели на базовой логистической регрессии

In [152]:
print(f'Accuracy: {np.mean(cv_tf_idf["test_accuracy"])}')
print(f'f1: {np.mean(cv_tf_idf["test_f1"])}')
print(f'roc_auc: {np.mean(cv_tf_idf["test_roc_auc"])}')

Accuracy: 0.9062543167381879
f1: 0.8176817759490026
roc_auc: 0.9764501648718398


### метрики уменьшились несильно

## Однако попробую подберу наилучшие гиперпараметры 

In [115]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [153]:
param_grid_for_logreg = [{'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'penalty': ['l2', 'l1'], 'solver': ['liblinear']}, 
              {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'penalty': ['l2'], 'solver': ['lbfgs', 'newton-cg']}]

In [154]:
gridsearch = GridSearchCV(LogisticRegression(max_iter=1000), 
                          param_grid=param_grid_for_logreg,
                         cv = 5,
                         )

In [155]:
gridsearch.fit(X_train_BOW_bi_select_features,y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000),
             param_grid=[{'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                          'penalty': ['l2', 'l1'], 'solver': ['liblinear']},
                         {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                          'penalty': ['l2'],
                          'solver': ['lbfgs', 'newton-cg']}])

In [156]:
gridsearch.best_params_

{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}

In [157]:
best_logreg_bow_model = gridsearch.best_estimator_

In [159]:
cv_best_logreg_bow = cross_validate(best_logreg_bow_model, X_train_BOW_bi_select_features, y_train, cv=5,scoring=['accuracy','f1','roc_auc'])

In [160]:
print(f'Accuracy: {np.mean(cv_best_logreg_bow["test_accuracy"])}')
print(f'f1: {np.mean(cv_best_logreg_bow["test_f1"])}')
print(f'roc_auc: {np.mean(cv_best_logreg_bow["test_roc_auc"])}')

Accuracy: 0.9444035635971121
f1: 0.9001029250843484
roc_auc: 0.9785512323170732


### Метрики немного улучшились

### Теперь попробую подобрать для модели с векторизацией TF-IDF

In [ ]:
print(5)

In [ ]:
gridsearch_tf_idf = GridSearchCV(LogisticRegression(max_iter=1000), 
                          param_grid=param_grid_for_logreg,
                         cv = 5,
                         )

In [ ]:
%time gridsearch_tf_idf.fit(X_train_TF_IDF_bi_select_features.loc[:,100],y_train)

In [128]:
gridsearch_tf_idf.best_params_

{'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}

In [129]:
best_logreg_tf_idf_model = gridsearch_tf_idf.best_estimator_

In [131]:
cv_best_logreg_tf_idf = cross_validate(best_logreg_tf_idf_model, X_train_TFIDF, y_train, cv=5,scoring=['accuracy','f1','roc_auc'])

In [132]:
print(f'Accuracy: {np.mean(cv_best_logreg_tf_idf["test_accuracy"])}')
print(f'f1: {np.mean(cv_best_logreg_tf_idf["test_f1"])}')
print(f'roc_auc: {np.mean(cv_best_logreg_tf_idf["test_roc_auc"])}')

Accuracy: 0.9354960993670671
f1: 0.8858835189802573
roc_auc: 0.9780317045678772


### Метрики здесь тоже хуже, поэтому в дальнейшем буду использовать только BOW

In [80]:
gridsearch.best_params_

{'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}

In [82]:
best_logreg = gridsearch.best_estimator_

In [83]:
best_logreg.fit(X_train,y_train)

LogisticRegression(C=100, max_iter=1000, solver='newton-cg')

In [84]:
y_pred = best_logreg.predict(X_train)

In [85]:
print(f'Accuracy: {accuracy_score(y_train,y_pred)}')
print(f'f1: {f1_score(y_train,y_pred)}')
print(f'roc_auc: {roc_auc_score(y_train,y_pred)}')

Accuracy: 0.9995699717410002
f1: 0.9992721222834563
roc_auc: 0.9994533257814456


### Теперь для наивного байесовского классификатора

#### Тк у него нет параметров для оптимизации, сделаю просто кросс-валидацию

In [86]:
from sklearn.naive_bayes import GaussianNB

In [90]:
cv_nb = cross_validate(GaussianNB(),
                       X_train.toarray(), 
                       y_train,
                       cv=5,
                       scoring=['accuracy','f1','roc_auc'])

In [91]:
print(f'Accuracy: {np.mean(cv_nb["test_accuracy"])}')
print(f'f1: {np.mean(cv_nb["test_f1"])}')
print(f'roc_auc: {np.mean(cv_nb["test_roc_auc"])}')

Accuracy: 0.7911292775808906
f1: 0.7025263193645348
roc_auc: 0.8037246117444748


#### Результаты получились хуже, чем у модели Логистической регрессии

### Теперь попробуем векторизировать наши данные с помощью BoW


In [110]:
from sklearn.feature_extraction.text import CountVectorizer

In [111]:
vectorizer = CountVectorizer()

In [113]:
# Преобразование предложений в векторы
X_train_bow = vectorizer.fit_transform(df['text'])

In [116]:
param_grid = [{'solver': ['lbfgs', 'newton-cg']},
              {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'penalty': ['l2', 'l1'], 'solver': ['liblinear']}, 
              {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'penalty': ['l2'], 'solver': ['lbfgs', 'newton-cg']}]

In [117]:
gridsearch = GridSearchCV(LogisticRegression(max_iter=1000), 
                          param_grid=param_grid,
                         cv = 5,
                         )

In [119]:
gridsearch.fit(X_train_bow,y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000),
             param_grid=[{'solver': ['lbfgs', 'newton-cg']},
                         {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                          'penalty': ['l2', 'l1'], 'solver': ['liblinear']},
                         {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                          'penalty': ['l2'],
                          'solver': ['lbfgs', 'newton-cg']}])

In [120]:
gridsearch.best_params_

{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}

In [121]:
best_logreg = gridsearch.best_estimator_

In [122]:
best_logreg.fit(X_train,y_train)

LogisticRegression(C=1, max_iter=1000, solver='liblinear')

In [123]:
y_pred = best_logreg.predict(X_train)

In [124]:
print(f'Accuracy: {accuracy_score(y_train,y_pred)}')
print(f'f1: {f1_score(y_train,y_pred)}')
print(f'roc_auc: {roc_auc_score(y_train,y_pred)}')

Accuracy: 0.9541098415038702
f1: 0.9188308160382483
roc_auc: 0.9323555313906748


#### Метрики ухудшились. Теперь проверю BoW с наивным байесовским классификатором

In [125]:
cv_nb_bow = cross_validate(GaussianNB(),
                       X_train_bow.toarray(), 
                       y_train,
                       cv=5,
                       scoring=['accuracy','f1','roc_auc'])

In [126]:
print(f'Accuracy: {np.mean(cv_nb_bow["test_accuracy"])}')
print(f'f1: {np.mean(cv_nb_bow["test_f1"])}')
print(f'roc_auc: {np.mean(cv_nb_bow["test_roc_auc"])}')

Accuracy: 0.7795795355472774
f1: 0.6986471536903992
roc_auc: 0.8042721421196516


#### Эта модель показала наилучшие результаты

### Я выбрал модель логистической регрессии и наивного байесовского классификатора, поскольку в данной статье: https://habr.com/ru/articles/768470/ приведены доказательства в пользу этих моделей в качестве наилучших моделей для классификации спама

##### По итогам, наилучшая модель классификации - модель логистической регрессии с параметрами:{'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}. А векторизация TF-IDF

In [127]:
the_best_model = LogisticRegression(max_iter=1000, C=100, penalty='l2',solver='newton-cg')

In [154]:
vectorizer = TfidfVectorizer()

In [166]:
#import test csv
df_test = pd.read_csv('test_spam.csv')

In [169]:
df['text'].shape, df_test['text'].shape

((16278,), (4070,))

In [170]:
X_test_train = pd.concat([df['text'],df_test['text']])

(20348,)

In [176]:
X_test_train_transformed = vectorizer.fit_transform(X_test_train)
X_test_train_transformed = X_test_train_transformed.toarray()

In [177]:
X_train = X_test_train_transformed[0:16278]
X_test = X_test_train_transformed[16278:]

In [179]:
y_train = df['text_type']

In [180]:
cv_best = cross_validate(the_best_model,X_train,y_train,scoring=['accuracy','f1','roc_auc'],cv=5)

KeyboardInterrupt: 

In [158]:
print(f'Accuracy: {np.mean(cv_best["test_accuracy"])}')
print(f'f1: {np.mean(cv_best["test_f1"])}')
print(f'roc_auc: {np.mean(cv_best["test_roc_auc"])}')

Accuracy: 0.9410867612480516
f1: 0.8954358501564433
roc_auc: 0.9820789994279085


In [159]:
#import test csv
df_test = pd.read_csv('test_spam.csv')

In [160]:
df_test.head()

,text
0,j jim whitehead ejw cse ucsc edu writes j you ...
1,original message from bitbitch magnesium net p...
2,java for managers vince durasoft who just taug...
3,there is a youtuber name saiman says
4,underpriced issue with high return on equity t...


In [164]:
X_test = vectorizer.fit_transform(df_test['text'])
X_test.shape

(4070, 23419)

In [165]:
y_pred = the_best_model.predict(X_test)

ValueError: X has 23419 features, but LogisticRegression is expecting 46622 features as input.